### Test dynamic rerouting service ###

In [103]:
import requests
import time
import base64
import pandas as pd
from tqdm import tqdm
import random

In [97]:
def base64_encode(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

sample_image = "sample_images/sample_cat.jpeg"
encoded_image = base64_encode(sample_image)
encoded_images = [encoded_image] * batch_size

sample_data = {"images": encoded_images}

In [100]:
def send_request(is_secure):
    start_time = time.time()
    url = base_url
    if is_secure:
        url = base_url + "predict-secure/"
    else:
        url = base_url + "predict-public/"
    response = requests.post(url, json=sample_data)
    response_time = time.time() - start_time
    return response.status_code, response_time

### Send groups of requests separately ###

In [102]:
public_times = []
secure_times = []

base_url = 'http://18.223.112.165/'  
num_public_requests = 10
num_secure_requests = 10
batch_size = 2

print('Sending requests...')

for _ in range(num_public_requests):
    status, time_taken = send_request(is_secure=False)
    public_times.append(time_taken)

for _ in range(num_secure_requests):
    status, time_taken = send_request(is_secure=True)
    secure_times.append(time_taken)

avg_public_time = sum(public_times) / len(public_times)
avg_secure_time = sum(secure_times) / len(secure_times)

print(f'Average response time for public requests: {avg_public_time:.4f} seconds.')
print(f'Average response time for secure requests: {avg_secure_time:.4f} seconds.')

Sending requests...
Average response time for public requests: 0.3290 seconds.
Average response time for secure requests: 0.3537 seconds.


### Send intermixed in a single loop ###

In [106]:
base_url = 'http://18.223.112.165/'  
num_public_requests = 20
num_secure_requests = 80
batch_size = 2

num_requests = num_public_requests + num_secure_requests
public_times = []
secure_times = []

prob_secure = num_secure_requests / num_requests

print(f'Sending {num_secure_requests} secure requests and {num_public_requests} public requests with batch size {batch_size}')

for _ in tqdm(range(num_requests), desc="Sending requests"):
    is_secure = random.random() < prob_secure  

    status, time_taken = send_request(is_secure=is_secure)
    if is_secure:
        secure_times.append(time_taken)
    else:
        public_times.append(time_taken)

avg_public_time = sum(public_times) / len(public_times) if public_times else 0
min_public_time = min(public_times) if public_times else 0
max_public_time = max(public_times) if public_times else 0

avg_secure_time = sum(secure_times) / len(secure_times) if secure_times else 0
min_secure_time = min(secure_times) if secure_times else 0
max_secure_time = max(secure_times) if secure_times else 0

print(f'Public Requests - Avg: {avg_public_time:.4f}, Min: {min_public_time:.4f}, Max: {max_public_time:.4f} seconds.')
print(f'Secure Requests - Avg: {avg_secure_time:.4f}, Min: {min_secure_time:.4f}, Max: {max_secure_time:.4f} seconds.')

Sending 80 secure requests and 20 public requests with batch size 2


Sending requests: 100%|██████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.81it/s]

Public Requests - Avg: 0.3154, Min: 0.2801, Max: 0.3635 seconds.
Secure Requests - Avg: 0.3655, Min: 0.3103, Max: 0.5740 seconds.
